## Prova amb vídeo del model entrenat amb la tècnica de la distància interpupil·lar

In [14]:
import cv2
import mediapipe as mp
import face_recognition
import joblib
import numpy as np
import pandas as pd

rf_classifier = joblib.load('models/svm/svm_classifier_eyes.joblib')
index_lips = [61, 76, 62, 78, 
              185, 184, 183, 191, 95, 96, 77, 146, 
              40, 74, 42, 80, 88, 89, 90, 91, 
              39, 73, 41, 81, 178, 179, 180, 181, 
              37, 72, 38, 82, 87, 86, 85, 84,
              0, 11, 12, 13, 14, 15, 16, 17,
              267, 302, 268, 312, 317, 316, 315, 314, 
              269, 303, 271, 311, 402, 403, 404, 405, 
              270, 304, 272, 310, 318, 319, 320, 321, 
              409, 408, 407, 415, 324, 325, 307, 375, 
              308, 292, 306, 291]
index_eyes = [159, 386]

In [16]:
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils

cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)

with mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, min_detection_confidence=0.5) as face_mesh:
    while True:
        ret, frame = cap.read()
        if ret == False: break
        frame = cv2.flip(frame, 1)
        height, width, _ = frame.shape

        # face_locations = face_recognition.face_locations(frame,)
        # if face_locations != []:
        #     for face_location in face_locations:
        #         cv2.rectangle(frame, (face_location[3], face_location[0]), (face_location[1], face_location[2]), (0, 255, 0), 2)
        lip_info = []
        eyes_info = []
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(frame_rgb)
        if results.multi_face_landmarks is not None:
            for face_landmarks in results.multi_face_landmarks:
                for index in index_lips:
                    lip_info.append([face_landmarks.landmark[index].x, face_landmarks.landmark[index].y])
                    x = int(face_landmarks.landmark[index].x * width)
                    y = int(face_landmarks.landmark[index].y * height)
                    cv2.circle(frame, (x, y), 1, (3, 187, 133), 1)
                for index in index_eyes:
                    eyes_info.append([face_landmarks.landmark[index].x, face_landmarks.landmark[index].y])
                    x = int(face_landmarks.landmark[index].x * width)
                    y = int(face_landmarks.landmark[index].y * height)
                    cv2.circle(frame, (x, y), 1, (3, 187, 133), 1)

        pos_x = []
        pos_y = []
        for i in lip_info:
            pos_x.append(i[0])
            pos_y.append(i[1])

        aux_x = np.mean(pos_x)
        aux_y = np.mean(pos_y)
        lip_info.append([aux_x, aux_y])

        lip_info = np.array(lip_info)
        eyes_info = np.array(eyes_info)
        
        coordenada_central = 80
        distancia_media_ojos = 63 #En milimetros
        distancia_ojos = np.linalg.norm(eyes_info[0] - eyes_info[1])
        escala_ojos = distancia_media_ojos / distancia_ojos
        
        img_lip_info_2 = []
        for i in range(len(lip_info) - 1):
            distancia = np.linalg.norm(lip_info[i] - lip_info[coordenada_central])
            img_lip_info_2.append(distancia * escala_ojos)
    
        lip_info = pd.DataFrame(img_lip_info_2).transpose()
        prediccion = rf_classifier.predict(lip_info)

        switch = {1: "Vocal A", 2: "Vocal E", 3: "Vocal I", 4: "Vocal O", 5: "Vocal U"}

        cv2.putText(frame, switch.get(prediccion[0]), (50, 150), 2, 0.7, (255, 0, 255), 2)
        cv2.imshow("Frame", frame)
        k = cv2.waitKey(1)
        if k == 27 & 0xFF: break
cap.release()
cv2.destroyAllWindows()